In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn.utils.multiclass import unique_labels

from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.utils import resample


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
#### read in the flattened dataframes with the labels for the 98 to 2015 data
MSFT_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_MSFT_98_15.csv")
AAPL_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AAPL_98_15.csv")
AMZN_df_98_to_15_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AMZN_98_15.csv")

In [3]:
###read in the flattened dataframes with the labels for 2015 to 2019
MSFT_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_MSFT_15_19.csv")
AAPL_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AAPL_15_19.csv")
AMZN_df_15_to_19_df = pd.read_csv(r"/Users/fahad/project_repo/data/external/multiclass_AMZN_15_19.csv")



In [59]:
#vertically stack the MSFT and AAPL dataframes on top of eachother
df = pd.concat([MSFT_df_98_to_15_df, AAPL_df_98_to_15_df], axis=0)
#vertically stack the MSFT&AAPL df and the AMZN dataframe ontop of eachother
df = pd.concat([df,AMZN_df_98_to_15_df],axis = 0)

df.Label.value_counts()
# #drop the dates column
# df = df.drop(['Unnamed: 0'], axis = 1)
# #store the labels as the y_train array object 
# y_train = df['Label']
# #drop the labels from the instances
# df = df.drop(['Label'], axis =1)
# #assign the remaining columns in the dataframe
# X_train = df


1    228
0    155
2    125
3    104
Name: Label, dtype: int64

In [5]:
df_2 = pd.concat([MSFT_df_15_to_19_df, AAPL_df_15_to_19_df], axis=0)
df_2 = pd.concat([df_2,AMZN_df_15_to_19_df],axis = 0)
dates = df_2['Unnamed: 0']

df_2 = df_2.drop(['Unnamed: 0'], axis = 1)

#get labels for the prediction data
y_test = df_2['Label']

#drop the labelss
df_2 = df_2.drop(['Label'], axis=1)

#X_test to predict the values
X_test = df_2

In [6]:
#prev 50 and 150
es = EarlyStopping(monitor='val_acc', mode='max', verbose=0, patience=100)
mc = ModelCheckpoint('1dconv_feature_engineered_model_chronological_multiclass.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
cb_list = [es,mc]

# X_Train

In [17]:
#list which will become the array of shape 
list_1 = list()
#transform each row into a (30,12) array append to the list
for index, row in df.iterrows():
    arr = np.array(row).reshape(30,12)
#     print (arr)
#     print (arr.shape)
    list_1.append(arr)

arr1 = np.asarray(list_1)

#arr1 is an array of all the arrays needed to utilise the 
X_train = arr1
    

X_train.shape

(612, 30, 12)

# X_test

In [8]:
list_2 = list()

for index, row in df_2.iterrows():
    arr_2 = np.array(row).reshape(30,12)
    list_2.append(arr_2)
    
arr2 = np.asarray(list_2)

X_test = arr2

X_test.shape

(150, 30, 12)

In [52]:
def make_timeseries_classifier(X_train, y_train, X_test, y_test):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.
    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).
    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    verbose, epochs, batch_size = 0, 10, 32
    
    #number of timesteps for each prediction is 30, number of features is 12, number of outputs is 1
    n_timesteps, n_features, n_outputs =30,12, 4
    
    model = Sequential()
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu',
    input_shape=(n_timesteps,n_features)))
    model.add(Convolution1D(filters=64, kernel_size=3, activation='relu')) 
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(4, activation='softmax')) 
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy']) # fit network
    
    return model

In [53]:
model_1 = make_timeseries_classifier(X_train, y_train, X_test, y_test)
verbose, epochs, batch_size = 0, 10, 32
model_1.summary()
print (model_1.inputs)

es = EarlyStopping(monitor='val_acc', mode='max', verbose=0, patience=75)
mc = ModelCheckpoint('1dconv_feature_engineered_model_chronological_multiclass.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
cb_list = [es,mc]





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 28, 64)            2368      
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 26, 64)            12352     
_________________________________________________________________
dropout_8 (Dropout)          (None, 26, 64)            0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 832)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               83300     
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 404       
Total para

In [54]:
model_1.fit(X_train, y_train, batch_size=128,
          validation_split=0.10, epochs=400
            , verbose=1, callbacks = cb_list)

Train on 550 samples, validate on 62 samples
Epoch 1/400
550/550 [==============================] - 1s 2ms/step - loss: 1.3853 - acc: 0.2709 - val_loss: 1.3323 - val_acc: 0.5323

Epoch 00001: val_acc improved from -inf to 0.53226, saving model to 1dconv_feature_engineered_model_chronological_multiclass.h5
Epoch 2/400
550/550 [==============================] - 0s 128us/step - loss: 1.3617 - acc: 0.3473 - val_loss: 1.2886 - val_acc: 0.5323

Epoch 00002: val_acc did not improve from 0.53226
Epoch 3/400
550/550 [==============================] - 0s 117us/step - loss: 1.3538 - acc: 0.3545 - val_loss: 1.2628 - val_acc: 0.5323

Epoch 00003: val_acc did not improve from 0.53226
Epoch 4/400
550/550 [==============================] - 0s 106us/step - loss: 1.3515 - acc: 0.3545 - val_loss: 1.2575 - val_acc: 0.5323

Epoch 00004: val_acc did not improve from 0.53226
Epoch 5/400
550/550 [==============================] - 0s 102us/step - loss: 1.3487 - acc: 0.3545 - val_loss: 1.2632 - val_acc: 0.5323



Epoch 00044: val_acc did not improve from 0.53226
Epoch 45/400
550/550 [==============================] - 0s 78us/step - loss: 1.0274 - acc: 0.5436 - val_loss: 1.1965 - val_acc: 0.4677

Epoch 00045: val_acc did not improve from 0.53226
Epoch 46/400
550/550 [==============================] - 0s 61us/step - loss: 1.0495 - acc: 0.5509 - val_loss: 1.2262 - val_acc: 0.5000

Epoch 00046: val_acc did not improve from 0.53226
Epoch 47/400
550/550 [==============================] - 0s 60us/step - loss: 1.0127 - acc: 0.5782 - val_loss: 1.2201 - val_acc: 0.4839

Epoch 00047: val_acc did not improve from 0.53226
Epoch 48/400
550/550 [==============================] - 0s 71us/step - loss: 1.0034 - acc: 0.5582 - val_loss: 1.1874 - val_acc: 0.5000

Epoch 00048: val_acc did not improve from 0.53226
Epoch 49/400
550/550 [==============================] - 0s 72us/step - loss: 0.9900 - acc: 0.5982 - val_loss: 1.1769 - val_acc: 0.5000

Epoch 00049: val_acc did not improve from 0.53226
Epoch 50/400
550/55

In [55]:
saved_model = load_model('1dconv_feature_engineered_model_chronological_multiclass.h5')



In [56]:
predictions = saved_model.predict_classes(X_test,batch_size = X_test.shape[0], verbose=1)

from sklearn.metrics import accuracy_score
print ("saved model acc: %s" % accuracy_score(y_test, predictions))
# accuracy: (tp + tn) / (p + n)

# precision tp / (tp + fp)
precision = precision_score(
    y_test,
    predictions,
    average=None)


recall = recall_score(y_test,
                      predictions, 
                     average=None)



150/150 [==============================] - 0s 2ms/step
saved model acc: 0.38


/Users/fahad/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
